In [1]:
import unsloth
from unsloth import FastLanguageModel
import torch
import os

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",          # Gemma 2.2x faster!
    "unsloth/Qwen2.5-3B-Instruct-bnb-4bit",
    "unsloth/Qwen2.5-Coder-7B-Instruct",
    "unsloth/Qwen2.5-3B-bnb-4bit",

] # More models at https://huggingface.co/unsloth

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-3B-bnb-4bit", # "unsloth/mistral-7b" for 16bit loading
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.3.15: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    NVIDIA GeForce RTX 4070. Num GPUs = 1. Max memory: 11.719 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      "embed_tokens", "lm_head",], # Add for continual pretraining
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2025.3.15 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


In [5]:
from owlready2 import get_ontology, Or

def read_owl_as_text(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            owl_content = file.read()
        return owl_content
    except FileNotFoundError:
        return "Error: OWL file not found."
    except Exception as e:
        return f"Error reading OWL file: {str(e)}"

# def load_and_read_owl(file_path):
#     result = []
#     try:
#         # Load the ontology from the OWL file
#         onto = get_ontology(file_path).load()
#
#         # Print basic information
#         print(f"Ontology loaded: {onto.base_iri}")
#         result.append(f"Ontology loaded: {onto.base_iri}")
#
#         # Access classes
#         print("\nClasses in the ontology:")
#         result.append("\nClasses in the ontology:")
#         for cls in onto.classes():
#             print(f" - {cls.name} ({cls.iri})")
#             class_info = f" - {cls.name} ({cls.iri})"
#             if cls.comment:  # Print comments if they exist
#                 print(f"   Comment: {cls.comment[0]}")
#                 class_info += f"\n   Comment: {cls.comment[0]}"
#             result.append(class_info)
#
#         # Access properties
#         print("\nObject properties in the ontology:")
#         result.append("\nObject properties in the ontology:")
#         for prop in onto.object_properties():
#             print(f" - {prop.name} ({prop.iri})")
#             prop_info = f" - {prop.name} ({prop.iri})"
#             if prop.comment:
#                 print(f"   Comment: {prop.comment[0]}")
#                 prop_info += f"\n   Comment: {prop.comment[0]}"
#             if prop.domain:
#                 print(f"   Domain: {prop.domain[0].name}")
#                 prop_info += f"\n   Domain: {prop.domain[0].name}"
#             if prop.range:
#                 print(f"   Range: {prop.range[0].name}")
#                 prop_info += f"\n   Range: {prop.range[0].name}"
#             if prop.inverse_property:
#                 print(f"   Inverse: {prop.inverse_property.name}")
#                 prop_info += f"\n   Inverse: {prop.inverse_property.name}"
#             result.append(prop_info)
#
#         result.append("\nData properties in the ontology:")
#         for data_prop in onto.data_properties():
#             data_info = f" - {data_prop.name} ({data_prop.iri})"
#             if data_prop.comment:
#                 data_info += f"\n   Comment: {data_prop.comment[0]}"
#             if data_prop.domain:
#                 data_info += f"\n   Domain: {data_prop.domain[0].name}"
#             if data_prop.range:
#                 data_info += f"\n   Range: {data_prop.range[0]}"
#             result.append(data_info)
#
#
#         # Return the ontology object for further use
#         return "\n".join(result)
#
#     except FileNotFoundError:
#         print("Error: OWL file not found.")
#         return None
#     except Exception as e:
#         print(f"Error loading OWL file: {str(e)}")
#         return None

In [6]:
import os
cd = os.getcwd()
owl_path = os.path.join(cd, "../Onto_mapping/OWLs/SOMA.owl")

In [22]:
soma_text_1 = read_owl_as_text(owl_path)
# soma_text_2 = load_and_read_owl(owl_path)

In [23]:
from datasets import Dataset, load_dataset, concatenate_datasets
data1 = {"text": soma_text_1}
datasets1 = Dataset.from_dict(data1)

In [9]:
datasets = concatenate_datasets([datasets1])

In [10]:
EOS_TOKEN = tokenizer.eos_token

In [11]:
def formatting_prompts_func(examples):
    return { "text" : [example + EOS_TOKEN for example in examples["text"]] }
datasets = datasets.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/699705 [00:00<?, ? examples/s]

In [28]:
# for row in datasets[0:1]["text"]:
#     print("=========================")
#     print(row)

print(data1)

{'text': '<?xml version="1.0"?>\n<rdf:RDF xmlns="http://www.ease-crc.org/ont/SOMA.owl#"\n     xml:base="http://www.ease-crc.org/ont/SOMA.owl"\n     xmlns:owl="http://www.w3.org/2002/07/owl#"\n     xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"\n     xmlns:xml="http://www.w3.org/XML/1998/namespace"\n     xmlns:xsd="http://www.w3.org/2001/XMLSchema#"\n     xmlns:SOMA="http://www.ease-crc.org/ont/SOMA.owl#"\n     xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#">\n    <owl:Ontology rdf:about="http://www.ease-crc.org/ont/SOMA.owl">\n        <owl:imports rdf:resource="http://www.ease-crc.org/ont/DUL.owl"/>\n        <owl:versionInfo>current</owl:versionInfo>\n    </owl:Ontology>\n    \n\n\n    <!-- \n    ///////////////////////////////////////////////////////////////////////////////////////\n    //\n    // Annotation properties\n    //\n    ///////////////////////////////////////////////////////////////////////////////////////\n     -->\n\n\n    \n\n\n    <!-- http://www.ease-crc.o

In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = datasets,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 8,

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,

        warmup_ratio = 0.1,
        num_train_epochs = 5,

        learning_rate = 5e-5,
        embedding_learning_rate = 5e-6,
        max_steps=20,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.00,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/699705 [00:00<?, ? examples/s]

In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 699,705 | Num Epochs = 1 | Total steps = 20
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 861,798,400/3,000,000,000 (28.73% trained)


Step,Training Loss
1,12.527300
2,12.733200
3,6.907000
4,2.781700
5,0.169100
6,0.001000
7,0.000000
8,0.000000
9,0.079800
10,0.018200


In [19]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 50

prompts = [
    "what is the class related to Placing in my SOMA ontology",
    # "what are the flanagan motion phases involved in the task- cut the apple. Give it in json format, no further explanation is needed",
    # "what are the motion constraints involved in the task- cut the apple. Give it in json format, no further explanation is needed",
    # "what are the framenet elements involved in the task- cut the apple. Give it in json format, no further explanation is needed",
    # "what are the object and tools involved in the task- cut the apple. Give it in json format, no further explanation is needed"
]

inputs = tokenizer(
[
    prompts[0]
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 2048,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()

length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

what is the class related to Placing in my SOMA
ontology<|endoftext|>